In [2]:
from selenium import webdriver
import time, re
import base64
from bs4 import BeautifulSoup

import psycopg2
conn = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="5432", database="postgres")
cur = conn.cursor()

import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server()
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
service = build('gmail', 'v1', credentials=creds)

def get_code():
    results = service.users().messages().list(userId='me',labelIds = ['INBOX']).execute()
    messages = results.get('messages', [])
    if not messages:
        print("No messages found.")
    else:
        # https://developers.google.com/gmail/api/v1/reference/users/messages?refresh=1
        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            body = msg['payload']['parts'][1]['body']['data']
            contents = base64.urlsafe_b64decode(body)
            soup = BeautifulSoup(contents, 'html.parser')
            return(int(soup.find_all('h2')[1].text.strip()))

driver = webdriver.Chrome()
# driver = webdriver.Firefox()

driver.get('https://www.freelancer.com/jobs/')
time.sleep(1)
login_button_1 = driver.find_element_by_xpath('/html/body/div[1]/header/div/div/div[2]/fl-login-signup-angular/a[1]')
login_button_1.click()

time.sleep(1)
var_username = 'rajatpundir13@gmail.com'
var_pass = 'R34p3r13'
username = driver.find_element_by_xpath('//*[@id="username"]')
password = driver.find_element_by_xpath('//*[@id="password"]')
username.clear()
username.send_keys(var_username)
password.clear()
password.send_keys(var_pass)
remember_me = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-login-form/div[2]/form[2]/fieldset/ol/li[3]/div/label')
# remember_me.clear()
remember_me.click()
login_button_2 = driver.find_element_by_xpath('//*[@id="login_btn"]')
login_button_2.click()
time.sleep(8)
give_me_code = driver.find_element_by_xpath('/html/body/fl-ui-modal/div/div[1]/div/div/fl-compiled/fl-login-signup-modal/div/div/fl-login/fl-totp-form/div[2]/div/form/fieldset/ol/li[2]/input')
give_me_code.send_keys(get_code())


In [ ]:
tree = {}
proj = set()
prev_count = 0

try:
    while True:
        rows = driver.find_element_by_xpath('//*[@id="project_table"]/tbody')
        soup = BeautifulSoup(rows.get_attribute('innerHTML'), 'html.parser')
        for i in range(len(soup.find_all('tr'))):
            row = soup.find_all('tr')[i]
            tags = []
            for j in range(len(row.find_all('span')[1].find_all('a'))):
                tags.append('\'' + row.find_all('span')[1].find_all('a')[j].text.strip() + '\'')
            url = row.find_all('a')[0]['href']
            project_id = int(row['id'])
            body = row.find_all('td')[0].find_all('p').pop().text.strip()
            bid_count = re.findall(r'\d+', row.find_all('td')[1].text.strip())[0]
            time_left = row.find_all('td')[2].find_all('small').pop().text
            money = row.find_all('td')[4].find_all('span')[0].text.strip()
            if project_id not in proj:
                proj.add(project_id)
                tree[project_id] = {}
            cur.execute("SELECT id FROM projects where id = %s;", (project_id,))
            if cur.fetchone() is None:
                tree[project_id]['tags'] = tags
                cur.execute("INSERT INTO projects (id, bid_count, url, body, time_left, money, time) VALUES (%s, %s, %s, %s, %s, %s, %s)", (project_id, bid_count, url, body, time_left, money, time.time()))
                conn.commit()
                cur.execute("INSERT INTO tags (id, tag) VALUES (%s, ARRAY[%s])", (project_id, ', '.join(tags)))
                print('new-'+str(project_id), bid_count, url, money)
            else:
                cur.execute("SELECT bid_count from projects where id=%s", (project_id,))
                r = cur.fetchone()
                if r[0] != int(bid_count):
                    cur.execute("UPDATE projects SET bid_count=%s, url=%s, body=%s, time_left=%s, money=%s WHERE id=%s", (bid_count, url, body, time_left, money, project_id))
                    cur.execute("SELECT bid_count, time, url from projects where id=%s", (project_id,))
                    r = cur.fetchone()
                    print(r[0], time.time() - r[1], r[2])
        conn.commit()
        cur.execute("SELECT count(*) from projects;")
        count = cur.fetchone()[0]
        if count != prev_count:
            print('count = ', count)
            prev_count = count
        driver.refresh()
        time.sleep(4)
except:
    conn.commit()
    print('success')

new-19400863 0 https://www.freelancer.com/projects/seo/key-words-rank-top-google-19400863/ ₹2087 - ₹17397
new-19400861 7 https://www.freelancer.com/projects/graphic-design/graphic-design-for-screen-printing-19400861/ ₹3201
new-19400860 0 https://www.freelancer.com/projects/audio-services/audio-recording-production/ ₹1739 - ₹3480/hr Unspecified
new-19400859 1 https://www.freelancer.com/projects/english-us/translate-english-documents/ ₹2088
new-19400858 1 https://www.freelancer.com/projects/internet-marketing/online-marketing-sell-house/ ₹3131
new-19400855 20 https://www.freelancer.com/projects/graphic-design/need-graphic-artist-design-some/ ₹1670 / hrUnspecified
new-1495221 0 https://www.freelancer.com/contest/moms-logo-1495221.html ₹1740
new-19400854 2 https://www.freelancer.com/projects/technical-support/help-with-windows-excel-problem/ ₹7717
new-19400853 3 https://www.freelancer.com/projects/articles/article-writer-required-urgently-big/ ₹265396
new-19400851 0 https://www.freelancer.